In [2]:
import sys
import os
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import scipy.signal as sig
import datetime as dt
import random as rand
from scipy import stats, interp
from IPython import embed
from shutil import copyfile, copy2
from load_intan_rhs_format import read_data

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix 
from sklearn import svm
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold

In [26]:
def calcmeans(df):
    means = pd.DataFrame()
    for i in range(0,16):
        singlech = df.iloc[:,np.add(i,np.multiply(16,range(0,int(df.shape[1]/16-1))))]
        means = pd.concat([means,np.mean(singlech, axis=1)],axis=1)
    means.columns = range(0,16)
    return means

In [45]:
def detrend(df,base):
    means = calcmeans(base)
    detrended = pd.DataFrame()
    for i in range(0,df.shape[1]):
        print('detrending seg ' + str(i+1) + ' of ' + str(df.shape[1]))
        ch = i % 16
        orig = df.iloc[:,i]
        mean = means[ch]
        new = orig - mean
        detrended = pd.concat([detrended,new],axis=1)
    return detrended

In [28]:
def calcfeats(df):
    d = {'max': np.max(df),
         'argmax': df.idxmax(),
         'min': np.min(df),
         'argmin': df.idxmin()
        }
    return pd.DataFrame(d)

In [29]:
# loading in previously computed 1second response traces to each stim for interictal
os.chdir('C:\\Users\\Senan\\SpontStimAnalysis\\Interictal\\RespData')
interictal01 = pd.read_pickle('sKH01interictal1s.pkl')
interictal03 = pd.read_pickle('sKH03interictal1s.pkl')

# loading preictal same
os.chdir('C:\\Users\\Senan\\SpontStimAnalysis\\Preictal\\RespData')
preictal01 = pd.read_pickle('sKH01preictal1s.pkl')
preictal03 = pd.read_pickle('sKH03preictal1s.pkl')

# loading sz times
sztimes01 = pd.read_pickle('sKH01sztimes.pkl')
sztimes03 = pd.read_pickle('sKH03sztimes.pkl')

In [ ]:
# detrend compute feats on sKH03 
# using the first 100 interictal responses to detrend all

# interdat03 = detrend(interictal03,interictal03.iloc[:,0:1600])

preicdat03 = detrend(preictal03, interictal03.iloc[:,0:1600])


detrending seg 1 of 16880
detrending seg 2 of 16880
detrending seg 3 of 16880
detrending seg 4 of 16880
detrending seg 5 of 16880
detrending seg 6 of 16880
detrending seg 7 of 16880
detrending seg 8 of 16880
detrending seg 9 of 16880
detrending seg 10 of 16880
detrending seg 11 of 16880
detrending seg 12 of 16880
detrending seg 13 of 16880
detrending seg 14 of 16880
detrending seg 15 of 16880
detrending seg 16 of 16880
detrending seg 17 of 16880
detrending seg 18 of 16880
detrending seg 19 of 16880
detrending seg 20 of 16880
detrending seg 21 of 16880
detrending seg 22 of 16880
detrending seg 23 of 16880
detrending seg 24 of 16880
detrending seg 25 of 16880
detrending seg 26 of 16880
detrending seg 27 of 16880
detrending seg 28 of 16880
detrending seg 29 of 16880
detrending seg 30 of 16880
detrending seg 31 of 16880
detrending seg 32 of 16880
detrending seg 33 of 16880
detrending seg 34 of 16880
detrending seg 35 of 16880
detrending seg 36 of 16880
detrending seg 37 of 16880
detrending

In [13]:
interfeat03 = calcfeats(interdat03)
prefeatskh03 = calcfeats(preicdat03)

interfeat03.head()

array([[-1.64339830e+03, -1.65755455e+01,  1.30842506e+03, ...,
        -1.65556150e+02, -1.53988216e+03,  5.66137713e+03],
       [-1.46871935e+03,  6.14038676e+01,  1.39667449e+03, ...,
        -7.86255454e+01, -1.36870761e+03,  5.42286841e+03],
       [-1.03001761e+03,  1.12422310e+02,  1.36749374e+03, ...,
        -1.11834666e+02, -1.33678235e+03,  5.23246427e+03],
       ...,
       [-3.35006409e+01, -1.56504703e+00, -1.31507605e+02, ...,
         3.89811892e+01,  6.78375751e+01,  3.36857497e+01],
       [-3.64005201e+01, -1.03073991e+00, -1.35308257e+02, ...,
         3.49884817e+01,  6.99753507e+01,  3.47724339e+01],
       [-4.44961383e+01, -1.79301817e+01, -1.42457611e+02, ...,
         3.02203385e+01,  6.52509803e+01,  3.59956768e+01]])

In [25]:
t = interictal03.iloc[:,0:1600]

int(t.shape[1]/16-1)

# np.add(0,np.multiply(16,range(0,t.shape[1]/16-1)))

# singlech = t.iloc[:,]

99

In [39]:
interdat03.head()

,0,1,2,3,4,5,6,7,8,9,...,6,7,8,9,10,11,12,13,14,15
0,-459.145541,48.661073,-464.763724,831.875839,-1601.562295,-12.528623,-179.316403,-7.372848,-60.602342,-5.472707,...,8544.693688,38.320994,4960.096679,685.180130,5735.184576,-53.575353,-955.327189,-102.168732,-1977.173704,9099.216407
1,-495.858702,21.326312,-489.445897,866.964509,-1340.784708,-22.022760,-282.711623,-12.652440,-11.275815,-37.228373,...,7869.775716,-2.933290,4623.595021,521.976390,5415.119902,-70.560433,-1019.182859,53.333672,-2005.976894,8857.345940
2,-361.953648,40.731060,-433.139875,724.977612,-951.263995,-17.996972,-248.271782,-9.251727,-19.414403,11.973476,...,6980.305664,-13.317119,4623.623502,375.470055,5106.454631,-66.748244,-1121.833451,-46.595985,-2199.077842,8536.996241
3,-289.197755,14.110662,-493.549960,699.173000,-788.198718,-17.946523,-148.202342,-0.258994,-50.291248,-21.551391,...,6556.739403,-7.018860,4488.673452,369.462095,4987.167653,-51.124106,-1197.241788,-6.985544,-2178.956159,8431.423764
4,-327.299944,14.641138,-444.095564,609.397437,-689.875189,-49.318642,-230.730047,-34.308394,-53.165920,1.961307,...,5854.099910,-20.895549,4397.756441,224.672618,4675.215724,-64.299526,-1236.351963,-3.444217,-2281.880018,8153.637133


In [38]:
interictal03.head()

,0,1,2,3,4,5,6,7,8,9,...,6,7,8,9,10,11,12,13,14,15
0,-1643.398301,-16.575546,1308.425064,1812.858175,-3568.588953,69.447808,-4541.488582,52.518931,-89.944118,-545.725727,...,4182.521509,98.212773,4930.754903,144.927110,4756.466494,-31.986172,-161.762296,-165.556150,-1539.882160,5661.377133
1,-1468.719347,61.403868,1396.674486,1825.386424,-2967.618806,76.865719,-4214.864706,54.375842,76.536539,-432.018099,...,3937.622633,64.094993,4711.407375,127.186664,4546.590867,-58.588637,-191.587859,-78.625545,-1368.707615,5422.868413
2,-1030.017614,112.422310,1367.493737,1615.549653,-2233.391422,90.325600,-3528.300886,80.774413,30.842877,-305.963019,...,3700.276560,76.709021,4673.880782,57.533561,4331.605826,-38.649800,-139.536184,-111.834666,-1336.782346,5232.464268
3,-873.584548,117.953181,1332.623153,1590.366700,-1905.425673,83.537480,-3095.135633,78.912874,73.127253,-262.774226,...,3609.806113,72.153009,4612.091953,128.239260,4285.470192,-51.992324,-206.902621,-149.746578,-1259.267087,5127.745862
4,-739.055003,139.326738,1357.967782,1423.064531,-1626.478908,48.878602,-2777.277891,45.236889,54.928362,-171.684560,...,3307.552066,58.649734,4505.850723,51.026751,4032.259843,-49.381990,-149.449217,-98.587419,-1194.055052,4914.462200


In [40]:
os.chdir('C:\\Users\\Senan\\SpontStimAnalysis\\Interictal\\RespData')
interdat03.to_pickle('interictal03_detrended.pkl')

In [41]:
test = pd.read_pickle('interictal03_detrended.pkl')

In [44]:
interdat01.head()

NameError: name 'interdat01' is not defined